In [ ]:
#Importing dependencies
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RNN, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
import random
import urllib.request  
import matplotlib.pyplot as plt
%matplotlib inline 
#For plotting the matplotlib graphs in notebook

## LAB1: Manual Sequential Model

In [ ]:
import pandas as pd
column_names = ['word1', 'word2', 'word3', 'word4']

gram2 = pd.read_csv('https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Love_Gram/love_gram.txt', delimiter='\t', names=column_names) #Importing csv file with column names
gram2 = gram2.drop(['word4'], axis=1) #Dropping fourth column as we will be using only the 1st 3 columns 
print("shape of data", gram2.shape)

In [ ]:
print("Few sample records from data \n", gram2.sample(10))
print("\nFrequency of word1 vlaues \n", gram2["word1"].value_counts())
print("\nFrequency of word2 vlaues \n", gram2["word2"].value_counts())

In [ ]:
"""
Finding our words to create dictionary
Here we find unique values in each column and save each of those values .
Later which we will take the unique value for the entire appened columns
This will be our vocabulary list,which are the unique words in our data file
"""
chars = []
for i in list(gram2.columns.values):
    for j in pd.unique(gram2[i]):
        chars.append(j)
chars = np.unique(chars)


print('Count of unique words overall:', len(chars))
print('unique words list:', chars)

In [ ]:
"""
creating our word:indice pair dictionary and inverse
Here will be creating two dictonary values
char_indices : This contains each words mapped to an unique digit 
indices_char : This contains each digits mapped to a word in the same sequence as char_indices 
"""
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print("char_indices dictionary \n",char_indices)
print("char_indices.keys \n", char_indices.keys())
print("char_indices.values \n", char_indices.values())
print("\n ########################################\n")
print("indices_char dictionary \n", indices_char)
print("indices_char keys \n",indices_char.keys())
print("indices_char values \n",indices_char.values())

In [ ]:
"""
Here the column in word1 are being mapped to digits,the first 10 words is shown as an example 
hate  => 41
hated => 42 and so on
"""
#### Onehot encoding of X
X1 = gram2['word1'].map(char_indices)
X1 = keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
print("X1.shape is ",X1.shape)

#Mapping y with indices
y1 = gram2['word2'].map(char_indices)
#Creatinng hot encodings for the final layer
#Creating one hot encoding ,meaning each digit is represented by 1's and 0's
#eg for 5 , 6 , 7 ,8 we represent them by [1,0,0,0] [0,1,0,0] [0,0,1,0] [0,0,0,1] these are the conventions used in frameworks
y1 = keras.utils.to_categorical(np.array(y1), num_classes=len(char_indices))
print("y1.shape is ",y1.shape)
#print(y1[0:5])

In [ ]:
#Lets take example of two words
print("The word is -->"+gram2['word1'][0])
print("The one hot encoded version of the word is \n",X1[0])
#print("The one hot encoded version of the digit {0} is \n{1}".format(X1[0],y1[0]))

print("\nThe word is --> "+gram2['word1'][500])
print("The one hot encoded version of the word is \n",X1[500])
#print("The one hot encoded version of the digit {0} is \n{1}".format(X1[30],y1[30]))

In [ ]:
"""
We will be creating a Simple Neural Network with 10 hidden nodes in each layer,whose activation functions are sigmoid 
and in the final layer we will be using softmax in the final layer 

Parameter count:
Here the parameters will be weights and bias 
for the first layer the input_dim is one ,thus there will be 10 weights and 10 biases,therefore a total of 20 in first layer
for the second layer the input dim is 139,thus there will be 1390 weights and 139 biases,a total of 1529 in second layer
therefore a total of 1549 parameters for the overall model 

"""

model1 = Sequential()

model1.add(Dense(10, input_dim=X1.shape[1], activation='sigmoid'))

model1.add(Dense(y1.shape[1] ,activation='softmax'))

model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = model1.fit(X1, y1, epochs=20, batch_size=50,  verbose=1)
# Print Accuracy
scores = model1.evaluate(X1, y1) 
print("%s: %.2f%%" % (model1.metrics_names[1], scores[1]*100))

In [ ]:
#We will see what the 1st hidden layer output representation of the data  
# to predict the hidden layer activations, 
# let's rewrite first layer of our model and give it the weights from fully trained previous model
model1h = Sequential()
model1h.add(Dense(10, input_dim=X1.shape[1], weights=model1.layers[0].get_weights()))
model1h.add(Activation('sigmoid'))

In [ ]:
# Getting the hidden layer activations
h1 = model1h.predict(X1)
#peak into our hidden layer activations
print("The hidden layer output for every record - Shape of it \n", h1.shape)
print("Few sample records from hidden layer \n",h1[:5])

In [ ]:
#h1 and word2 as out input, word3 as our y variable
#build model [X: 10 : 139(word3)]
#Pridict the final results
X2_2 = gram2['word2'].map(char_indices)
X2_2 = keras.utils.to_categorical(np.array(X2_2), num_classes=len(char_indices))
X2_2.shape

In [ ]:
"""
We append the input words of the words2 column in the output of the h1 layer,this gives us the combined input representation
"""
X2 = np.append(h1,X2_2, axis=1)
print("X2 Shape", X2.shape)
#print(X2[:5])

In [ ]:
#Mapping y2 with indices
y2 = gram2['word3'].map(char_indices)
#Creatinng hot encodings for the final layer
y2 = keras.utils.to_categorical(np.array(y2), num_classes=len(char_indices))
print("y2.shape", y2.shape)
#print(y2[:2])

In [ ]:
model2 = Sequential()

model2.add(Dense(10, input_dim=X2.shape[1], activation='sigmoid'))

model2.add(Dense(y2.shape[1], activation='softmax'))

model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train model
history = model2.fit(X2, y2, epochs=20, batch_size=50,  verbose=1)
# Print Accuracy
scores = model2.evaluate(X2, y2) 
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

In [ ]:
# A predict function that takes input word1 and word2; and predict word3 
#1. take the input word , and represent them using digits from the char_indices dictonary values
#2. getting the intermediate hidden nodes for word1
#3. appending hidden activations with word2 as final test set
#4. prediction on this test set
def two_step_pred(words_in):
    #indices_in = [char_indices[i] for i in words_in] #converting or words into indices
    
    index_input=char_indices[words_in[0]]
    indices_in = keras.utils.to_categorical(index_input, num_classes=len(char_indices))
    indices_in=indices_in.reshape(1,len(char_indices))
    h1_test = model1h.predict(indices_in) # getting our intermediate hidden activations from model1h
    
    
    index_input2=char_indices[words_in[1]]
    indices_in2 = keras.utils.to_categorical(index_input2, num_classes=len(char_indices))
    indices_in2= indices_in2.reshape(1,len(char_indices))
    X2_test = np.append(h1_test, indices_in2, axis=1) #preparing final test data by appending hidden with word2
    
    
    yhat = np.argmax(model2.predict(X2_test), axis=-1) #predicting final output from model2
    return indices_char[yhat[0]]

In [ ]:
print(two_step_pred(['love', 'it']))

In [ ]:
print(two_step_pred(['love', 'to']))

In [ ]:
print(two_step_pred(['love', 'the']))

## LAB: RNN for word prediction(Love gram data)

In [ ]:
#Preparing our data for model
#We take two words columns from the csv file and then convert them to digits 
X3 = gram2[['word1','word2']]
for i in list(X3.columns.values):
    X3[i] = X3[i].map(char_indices)

X3=np.array(X3)
#print(X3)
#The same data is reshaped with similar structure but appended with 1 value to make it 3de array
X3=np.reshape(X3,(X3.shape[0],2,1))
#print(X3)
X3 = keras.utils.to_categorical(np.array(X3), num_classes=len(char_indices))
print(X3.shape)
#print(X3)

In [ ]:
y3 = gram2['word3'].map(char_indices)
y3 = keras.utils.to_categorical(np.array(y3), num_classes=len(char_indices))
y3.shape

In [ ]:
print("time stamps in this model" , X3.shape[1])
print("one hot Variable" , X3.shape[2])

In [ ]:
model3 = Sequential()

#model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, input data_dim)))
model3.add(SimpleRNN(30, input_shape=(X3.shape[1],X3.shape[2])))

model3.add(Dense(len(char_indices), activation='softmax' ))
#model3.summary()

In [ ]:
# compile network
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model3.fit(X3, y3, epochs=20,  validation_data=(X3, y3))

In [ ]:
#Save Weights
model3.save_weights("Weights_trained.hdf5")
# load weights

urllib.request.urlretrieve("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Saved_models/RNN_Weights_trained_v10.hdf5", "RNN_Weights_trained_v10.hdf5")
model3.load_weights("RNN_Weights_trained_v10.hdf5")

#weightsfile= "D:\\Google Drive\\Training\\Datasets\\model_weights\\RNN_Weights_trained_v10.hdf5"
#model3.load_weights(weightsfile)

In [ ]:
# compile network
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model3.fit(X3, y3, epochs=10, verbose=1, validation_data=(X3, y3))

In [ ]:
# evaluate
# 1. Enocde the words to their resepctive digits value
# 2. Reshape according to input value trained to the network
# 3. Pass the value to the model,convert into integer while getting the word from indices_char dicitonary

def rnn_word_pred(in_text):
    print("Input is - " , in_text)
    encoded = [char_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded =keras.utils.to_categorical(np.array(encoded), num_classes=len(char_indices))
    yhat = np.argmax(model3.predict(encoded), axis=-1)[0]
    print("Output is --> " ,indices_char[yhat])

In [ ]:
rnn_word_pred(["love","the"])

In [ ]:
rnn_word_pred(["love","it"])

In [ ]:
rnn_word_pred(["love","to"])

In [ ]:
rnn_word_pred(["hate","the"])

## LSTM

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/12_Chars_Data/3Gram_12Chars.csv", "3Gram_12Chars.csv")

In [ ]:
text_read = open('3Gram_12Chars.csv').read().lower()
print("Text\n", text_read[0:300])
#Replace comma with space
text= text_read.replace(',',' ').replace('\r','')
#Unique characters in our dataset we then sort it
chars = sorted(list(set(text)))
#print("Unique Characters in the text \n ",chars)
#\n is character string for new line, we dont need that in our dictionary of chars
chars.remove('\n')
#print("\n Character after removing \'\\n\'",chars)
#print("\n All chars lenght", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
#print("characters to indices dictionary\n", char_indices)
indices_char = dict((i, c) for i, c in enumerate(chars))
#print("indices to char dictionary\n", indices_char)
#print('unique chars: ', {len(chars)})
data = text.splitlines()
#print(data[:20])
#Adding a space at the end
data = [i+' ' for i in data]
#print(data[0:200])
sentences = [[char_indices[j] for j in i] for i in data ]
Seq_ln = 14
X = []
y = []
for i in sentences:
    for j in range(len(i)-Seq_ln):
        X.append(i[j:j+Seq_ln])
        y.append(i[j+Seq_ln])


In [ ]:
#The first row is the X's first row up to 14 character
#The second row is the X's first row starting from second character up to 14 character
#The third row is the X's first row starting from third character up to 14 character and so on 
X=np.array(X)
X1=np.reshape(X,(X.shape[0],X.shape[1],1))
#print(X1.shape)
X1=keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
#print(X1.shape)

#Target Variable
#y[:10]
#Reshapig our label for model
y1 = np.array(y)
# one hot encode outputs
y1 = keras.utils.to_categorical(np.array(y), num_classes=len(char_indices))
#y1.shape

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.20)

In [ ]:
#building the model
model_LSTM = Sequential()
#model1.add(LSTM('number of hidden nodes in each cell', input_shape=(timesteps, data_dim)))
model_LSTM.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model_LSTM.add(Dense(len(char_indices) , activation='softmax'))
model_LSTM.summary()

In [ ]:
# compile network
model_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_LSTM.fit(X_train, y_train, epochs=1, verbose=1, validation_data=(X_test, y_test))
model_LSTM.save_weights("char_LSTM_weights.hdf5")

In [ ]:
# load weights from a pre-trained model
urllib.request.urlretrieve("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Saved_models/char_LSTM_model_weights_v1.hdf5", "char_rnn_model_weights_LSTM_v1.hdf5")
model_LSTM.load_weights("char_rnn_model_weights_LSTM_v1.hdf5")

## Predictions

In [ ]:
#function to prepare test input
def prepare_input1(in_text):
    X1 = np.array([char_indices[i] for i in in_text]).reshape(1,14,1)
    X1= keras.utils.to_categorical(np.array(X1), num_classes=len(char_indices))
    return(X1)
#function to loop our preditions
def complete_pred1(in_text):
    #original_text = in_text
    #generated = in_text
    completion = ''
    while True:
        x = prepare_input1(in_text)
        pred = np.argmax(model_LSTM.predict(x, verbose=0), axis=-1)[0]
        next_char = indices_char[pred]

        in_text = in_text[1:] + next_char
        completion += next_char

        if len(completion)> 20 or next_char == ' ':
            return completion

In [ ]:
in_text = 'whatever they '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'these results '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'in accordance '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'knowledge and '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'political and '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)
in_text = 'of particular '
out_word = complete_pred1(in_text)
print("Input text -->", in_text, "; predicted output ---> ", out_word)